**Een Identifier die gebruik maakt van active learning. 500 vragen zijn zelf gelabeled. Vervolgens fine tunen we een bert model hierop. We laten deze getunede bert dan zelf 1000 ongeziene vragen labelen. We kijken naar de 20% waar het model het minst zeker van is. Die kijken we handmatig na. De gecorrigeerde voegen we dan toe aan de training data en zo herhalen we dit proces tot het bert model naar behoren werkt.** 

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load labeled data
df = pd.read_excel("Trainig_data.xlsx")  
df.columns = ['question', 'label']  

# Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [3]:
'''
Een random lange zin om te zien of we met 128 tokens per zin goed zitten. Deze zin bevat 44 dus 128 zou ok moeten zijn 
'''


tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
tokenizer2 = BertTokenizer.from_pretrained("bert-base-multilingual-cased" )
tokenizer3 = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

text = "Zal de minister initiatieven nemen om ervoor te zorgen dat personen in een palliatief zorgtraject in de toekomst wel aanspraak kunnen maken op een tegemoetkoming van hulpmiddelen? Zo ja, welke en binnen welke termijn?"

tokens = tokenizer.tokenize(text)
tokens2 = tokenizer2.tokenize(text)
tokens3 = tokenizer3.tokenize(text)

print("Number of tokens:", len(tokens))
print("Tokens:", tokens)

print("Number of tokens:", len(tokens2))
print("Tokens:", tokens2)

print("Number of tokens:", len(tokens3))
print("Tokens:", tokens3)

Number of tokens: 44
Tokens: ['Zal', 'de', 'minister', 'initiatie', '##ven', 'nemen', 'om', 'ervoor', 'te', 'zorgen', 'dat', 'personen', 'in', 'een', 'pal', '##lia', '##tief', 'zorgt', '##ra', '##ject', 'in', 'de', 'toekomst', 'wel', 'aan', '##spraak', 'kunnen', 'maken', 'op', 'een', 'tegemoetkoming', 'van', 'hulpmiddel', '##en', '?', 'Zo', 'ja', ',', 'welke', 'en', 'binnen', 'welke', 'termijn', '?']
Number of tokens: 55
Tokens: ['Za', '##l', 'de', 'minister', 'init', '##iati', '##even', 'nemen', 'om', 'ervoor', 'te', 'zo', '##rgen', 'dat', 'personen', 'in', 'een', 'pal', '##lia', '##tief', 'zo', '##rgt', '##raj', '##ect', 'in', 'de', 'toe', '##komst', 'wel', 'aan', '##spraak', 'kunnen', 'maken', 'op', 'een', 'te', '##gem', '##oe', '##tkom', '##ing', 'van', 'hulp', '##mid', '##delen', '?', 'Zo', 'ja', ',', 'welke', 'en', 'binnen', 'welke', 'term', '##ijn', '?']
Number of tokens: 40
Tokens: ['Zal', 'Ġde', 'Ġminister', 'Ġinitiatieven', 'Ġnemen', 'Ġom', 'Ġervoor', 'Ġte', 'Ġzorgen', 'Ġdat'

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')  # or 'GroNLP/bert-base-dutch-cased'

def tokenize(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt", max_length=128
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)


In [30]:
from transformers import BertTokenizer
def remove_hoeveel(text):
    return ' '.join([w for w in text.split() if w.lower() != 'hoeveel'])

train_texts = [remove_hoeveel(t) for t in train_texts]
val_texts = [remove_hoeveel(t) for t in val_texts]


tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')  # or 'GroNLP/bert-base-dutch-cased'

def tokenize(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt", max_length=128
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)

In [31]:

class QuestionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = QuestionDataset(train_encodings, train_labels)
val_dataset = QuestionDataset(val_encodings, val_labels)


In [32]:
model = BertForSequenceClassification.from_pretrained('GroNLP/bert-base-dutch-cased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.445570
2,No log,0.488587
3,0.367900,0.633018


TrainOutput(global_step=717, training_loss=0.30861406845028927, metrics={'train_runtime': 105.2333, 'train_samples_per_second': 54.393, 'train_steps_per_second': 6.813, 'total_flos': 376511920220160.0, 'train_loss': 0.30861406845028927, 'epoch': 3.0})

In [33]:
print("hoeveel" in ' '.join(train_texts).lower())  # Should print False


True


In [7]:
import torch
import torch.nn.functional as F
import pandas as pd

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load data
unlabeled_df = pd.read_excel("unlabeled_questions.xlsx")
questions = unlabeled_df['question'].tolist()
encodings = tokenize(questions)
encodings = {k: v.to(device) for k, v in encodings.items()}

# Run model on the data
with torch.no_grad():
    outputs = model(**encodings)
    probs = F.softmax(outputs.logits, dim=1)
    
    predicted_probs, predicted_labels = torch.max(probs, dim=1)
    uncertainty = 1 - predicted_probs  # lower confidence = more uncertain

# Add results to DataFrame
unlabeled_df['predicted_label'] = predicted_labels.cpu().numpy()
unlabeled_df['confidence'] = predicted_probs.cpu().numpy()
unlabeled_df['uncertainty'] = uncertainty.cpu().numpy()

# Sort by uncertainty and get top 20%
top_uncertain = unlabeled_df.sort_values(by='uncertainty', ascending=False).head(int(0.2 * len(unlabeled_df)))

# Save to Excel for manual labeling
top_uncertain.to_excel("to_label_eng.xlsx", index=False)



FileNotFoundError: [Errno 2] No such file or directory: 'unlabeled_questions.xlsx'

In [34]:
# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']

# Clean and convert to string
df_test = df_test.dropna(subset=['question', 'label'])
df_test['question'] = df_test['question'].astype(str)

# Extract questions and labels
test_texts = df_test['question'].tolist()
test_labels = df_test['label'].tolist()

# Tokenize
test_encodings = tokenizer(
    test_texts, padding=True, truncation=True, return_tensors="pt", max_length=128
)


In [35]:
from transformers import Trainer

test_dataset = QuestionDataset(test_encodings, test_labels)

# Assuming you already have a `trainer` object that was used for training
trainer.evaluate(test_dataset)


{'eval_loss': 0.6006466746330261,
 'eval_runtime': 2.1834,
 'eval_samples_per_second': 228.54,
 'eval_steps_per_second': 28.854,
 'epoch': 3.0}

In [36]:
import numpy as np
from sklearn.metrics import classification_report

# Get raw predictions
outputs = trainer.predict(test_dataset)
preds = np.argmax(outputs.predictions, axis=1)

# Print evaluation report
print(classification_report(test_labels, preds))


              precision    recall  f1-score   support

           0       0.84      0.97      0.90       362
           1       0.86      0.50      0.64       137

    accuracy                           0.84       499
   macro avg       0.85      0.74      0.77       499
weighted avg       0.84      0.84      0.83       499



In [37]:
import pandas as pd
import re
from sklearn.metrics import classification_report

# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']
df_test = df_test.dropna(subset=['question'])
df_test['question'] = df_test['question'].astype(str)

# Your regex sets
statistical_sets = {
    "Set 1": [
        r"\b(hoeveel|aantal|percentage van|percentage|cijfer over|data over|statistieken van)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van)?\s*(de|een)?\s*(gegevens|statistieken|cijfers)\b"
    ],
    "Set 1.1": [
        r"\b(aantal|percentage van|percentage|cijfer over|data over|statistieken van)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van)?\s*(de|een)?\s*(gegevens|statistieken|cijfers)\b"
    ],
    "Set 2": [
        r"\b(hoe vaak|hoe groot|gemiddelde van|mediaan van|ratio van|procent van)\b",
        r"\b(stijging van|daling van|verandering in|ontwikkeling in|schommeling van|impact op)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|analyse van)?\s*(de|een)?\s*(cijfers|gegevens|statistieken|tellingen)\b"
    ],
    "Set 3": [
        r"\b(bruto|netto|inkomen|uitgaven|begroting|subsidies|belasting|tarief|BBP|inflatie|schulden|werkloosheid|bestedingen|consumptie)\b",
        r"\b(bevolking|leeftijdsgroep|demografie|migratie|emigratie|immigratie|huishoudens|gezinnen|verhouding tussen|dichtheid)\b",
        r"\b(aantal|hoeveelheid|grootte van|gemiddelde|mediaan|percentage|spreiding|percentiel|kwartiel|standaarddeviatie)\b"
    ],
    "Set 4": [
        r"\b(zorgkosten|patiënten|ziekenhuisopnames|sterftecijfers|levensverwachting|gezondheidsuitgaven|vaccinaties|epidemieën|medicatiegebruik)\b",
        r"\b(reistijd|filedruk|kilometers afgelegd|verkeersongevallen|CO2-uitstoot|luchtvervuiling|hernieuwbare energie|klimaatverandering|waterkwaliteit)\b",
        r"\b(grondprijzen|woningmarkt|huurprijzen|hypotheken|verkoopcijfers|bouwvergunningen|energieverbruik)\b"
    ],
    "Set 5": [
        r"\b(vergeleken met|ten opzichte van|in vergelijking met|in het verleden|sinds \d{4}|tussen \d{4} en \d{4})\b",
        r"\b(ontwikkeling sinds|historische gegevens|trendanalyse|jaarverslagen|statistische rapporten)\b"
    ],
    "Set 6": [
        r"\b(hoeveel|aantal|percentage|statistieken|cijfers|gegevens|data)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(per\s+\w+|per\s+\d+|in\s+\d{4}|tussen\s+\d{4}\s+en\s+\d{4})\b",  
        r"\b(wat is het aantal|hoe groot is|hoe vaak|gemiddelde van|ratio van)\b"
    ],
    "Set 7": [
        r"^(Wat is het aantal|Hoeveel)\b",
        r"\b(totale budget|welk budget|eenzelfde hoogte|gemiddeld aantal|geëvolueerd in de periode|jaarlijkse kostprijs |het aantal)\b",
        r"\b(bedroeg|welk|Kan de minister|wat was)\s*(de|een)?\s*(factuur|budget|overzicht|kostprijs)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|bedroeg)?\s*(de|een)?\s*(gegevens|statistieken|cijfers|factuur|overzicht)\b"
    ],
    "Set 8": [
        r"^(Wat is het aantal|Hoeveel)\b",
        r"\b(totale budget|welk budget|cijfer over|eenzelfde hoogte|gemiddeld aantal|geëvolueerd in de periode|jaarlijkse kostprijs|het aantal)\b",
        r"\b(bedroeg|welk|Kan de minister|wat was)\s*(de|een)?\s*(factuur|budget|overzicht|kostprijs)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|bedroeg)?\s*(de|een)?\s*(gegevens|statistieken|cijfers|factuur|overzicht)\b"
    ],
    "Set 2": [
        r"^(Hoeveel)\b"
    ]
}

# Function to apply regex rules to each question
def matches_any(question, patterns):
    return any(re.search(pat, question, flags=re.IGNORECASE) for pat in patterns)

# Evaluate each regex set
for set_name, patterns in statistical_sets.items():
    df_test[f"{set_name}_match"] = df_test['question'].apply(lambda q: int(matches_any(q, patterns)))

    if 'label' in df_test.columns:
        print(f"\n🧪 {set_name} — Evaluation against true labels:")
        print(classification_report(df_test['label'], df_test[f"{set_name}_match"], digits=3))



🧪 Set 1 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.914     0.914     0.914       362
           1      0.774     0.774     0.774       137

    accuracy                          0.876       499
   macro avg      0.844     0.844     0.844       499
weighted avg      0.876     0.876     0.876       499


🧪 Set 1.1 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.735     0.936     0.824       362
           1      0.395     0.109     0.171       137

    accuracy                          0.709       499
   macro avg      0.565     0.523     0.498       499
weighted avg      0.642     0.709     0.645       499


🧪 Set 2 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.728     0.961     0.829       362
           1      0.333     0.051     0.089       137

    accuracy                          0.711       4

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Make sure you have NLTK stopwords downloaded:
# import nltk; nltk.download('stopwords')

dutch_stopwords = stopwords.words('dutch')

vectorizer = TfidfVectorizer(max_features=5000, stop_words=dutch_stopwords)


# Load data
df_train = pd.read_excel("Trainig_data.xlsx")
df_test = pd.read_excel("Testset.xlsx")
df_train.columns = df_test.columns = ['question', 'label']
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# Vectorize

X_train = vectorizer.fit_transform(df_train['question'])
y_train = df_train['label']
X_test = vectorizer.transform(df_test['question'])
y_test = df_test['label']

# Models with your parameters
models = {
    "Naive Bayes": MultinomialNB(alpha=0.01),
    "Logistic Regression": LogisticRegression(
        C=10, class_weight='balanced', penalty='l2', solver='saga', max_iter=1000
    ),
    "SVM": SVC(C=1, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(
        class_weight='balanced', max_depth=None, n_estimators=250
    ),
    "XGBoost": XGBClassifier(
        max_depth=4, learning_rate=0.3, n_estimators=250,
        use_label_encoder=False, eval_metric='logloss'
    )
}

# Train and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\n🧪 {name}:\n")
    print(classification_report(y_test, preds, digits=3))



🧪 Naive Bayes:

              precision    recall  f1-score   support

           0      0.786     0.956     0.863       362
           1      0.729     0.314     0.439       137

    accuracy                          0.780       499
   macro avg      0.758     0.635     0.651       499
weighted avg      0.771     0.780     0.746       499


🧪 Logistic Regression:

              precision    recall  f1-score   support

           0      0.929     0.936     0.933       362
           1      0.828     0.810     0.819       137

    accuracy                          0.902       499
   macro avg      0.879     0.873     0.876       499
weighted avg      0.901     0.902     0.901       499


🧪 SVM:

              precision    recall  f1-score   support

           0      0.899     0.959     0.928       362
           1      0.867     0.715     0.784       137

    accuracy                          0.892       499
   macro avg      0.883     0.837     0.856       499
weighted avg      0.890

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\xgboost\core.py:158: UserWarning: [11:24:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🧪 XGBoost:

              precision    recall  f1-score   support

           0      0.902     0.964     0.932       362
           1      0.884     0.723     0.795       137

    accuracy                          0.898       499
   macro avg      0.893     0.843     0.864       499
weighted avg      0.897     0.898     0.894       499



In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Make sure you have NLTK stopwords downloaded:
# import nltk; nltk.download('stopwords')

dutch_stopwords = stopwords.words('dutch')

vectorizer = TfidfVectorizer(max_features=5000, stop_words=dutch_stopwords)


# Load data
df_train = pd.read_excel("Trainig_data.xlsx")
df_test = pd.read_excel("Testset.xlsx")
df_train.columns = df_test.columns = ['question', 'label']
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# ✅ Remove 'hoeveel'
def remove_hoeveel(text):
    return ' '.join([w for w in text.split() if w.lower() != 'hoeveel'])

df_train['question'] = df_train['question'].apply(remove_hoeveel)
df_test['question'] = df_test['question'].apply(remove_hoeveel)


# Vectorize

X_train = vectorizer.fit_transform(df_train['question'])
y_train = df_train['label']
X_test = vectorizer.transform(df_test['question'])
y_test = df_test['label']

# Models with your parameters
models = {
    "Naive Bayes": MultinomialNB(alpha=0.01),
    "Logistic Regression": LogisticRegression(
        C=10, class_weight='balanced', penalty='l2', solver='saga', max_iter=1000
    ),
    "SVM": SVC(C=1, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(
        class_weight='balanced', max_depth=None, n_estimators=250
    ),
    "XGBoost": XGBClassifier(
        max_depth=4, learning_rate=0.3, n_estimators=250,
        use_label_encoder=False, eval_metric='logloss'
    )
}

# Train and evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\n🧪 {name}:\n")
    print(classification_report(y_test, preds, digits=3))



🧪 Naive Bayes:

              precision    recall  f1-score   support

           0      0.772     0.961     0.856       362
           1      0.708     0.248     0.368       137

    accuracy                          0.766       499
   macro avg      0.740     0.605     0.612       499
weighted avg      0.754     0.766     0.722       499


🧪 Logistic Regression:

              precision    recall  f1-score   support

           0      0.841     0.906     0.872       362
           1      0.688     0.547     0.610       137

    accuracy                          0.808       499
   macro avg      0.765     0.727     0.741       499
weighted avg      0.799     0.808     0.800       499


🧪 SVM:

              precision    recall  f1-score   support

           0      0.798     0.961     0.872       362
           1      0.778     0.358     0.490       137

    accuracy                          0.796       499
   macro avg      0.788     0.659     0.681       499
weighted avg      0.793

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\xgboost\core.py:158: UserWarning: [11:31:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🧪 XGBoost:

              precision    recall  f1-score   support

           0      0.794     0.956     0.867       362
           1      0.746     0.343     0.470       137

    accuracy                          0.788       499
   macro avg      0.770     0.649     0.669       499
weighted avg      0.781     0.788     0.758       499

